In [97]:
import requests
import json
from dotenv import load_dotenv
import os
import dspy

In [102]:
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [70]:
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": "perplexity/sonar-small-chat", # Optional
    "messages": [
      {"role": "user", "content": "Give me links to current news articles related to this tweet: January 6th was a dark day for America. We must never forget the insurrection." \
        + "Make sure to format the output so that there is a list of dictionaries, with each dictionaries containing a title and a link to the article. Only return with the object in string format and no other text at all outside of it. \
          Do not have the string 'json' in the output"}
    ]
  })
)
data = json.loads(response.text)
message_content = data['choices'][0]['message']['content']
# lines = message_content.split('\n\n')
# out = json.dumps(lines)
message_content


'You can ignore this.'

In [34]:
entries

[{'title': 'Biden Vows to Bail Out Struggling Former Coronavirus Hotspots',
  'link': 'https://news.bloomberglaw.com/coronavirus-covid-19/biden-vows-to-bail-out-struggling-former-coronavirus-hotspots'},
 {'title': 'Donald Trump, 55 Other Lawmakers Testify They Were Treated Badly in Capitol Strife',
  'link': 'https://www.nytimes.com/2021/06/30/us/politics/trump-testimony-capitol-riot.html'},
 {'title': "Trump's CPAC Speech Oozes of 2020, Offers Little About 2024",
  'link': 'https://www.politico.com/news/2021/07/11/trump-cpac-speech-495378'},
 {'title': 'US Military to Leave Afghanistan by September 11',
  'link': 'https://www.npr.org/2021/06/19/1008715273/us-military-to-leave-afghanistan-by-september-11'},
 {'title': 'Biden Finds Bipartisan Compromise on Borders, Major Investment Roads and Bridges',
  'link': 'https://www.politico.com/news/2021/07/06/biden-republican-compromise-494754'}]

In [26]:
#loop through indexes 2, 4, 6, 8, 10:
for i in range(2, len(lines), 2):
    title = lines[i]
    link = lines[i+1]
    print(f"{title}: {link}")

In [138]:
def text_to_politics(text):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        },
        data=json.dumps({
            "model": "perplexity/sonar-small-chat", # Optional
            "messages": [
            {"role": "user", "content": "Give me the top three links to current news articles related to this tweet: " + text \
                + "Make sure to format the output so that there is a list of dictionaries, with each dictionaries containing a title and a link to the article." \
                    + "the output should be in json format"}
            ]
        })
    )
    data = json.loads(response.text)
    message_content = data['choices'][0]['message']['content']


    

    class classify_party(dspy.Signature):
        """Given tweet text, classify the political party of the author"""
        text = dspy.InputField()
        party = dspy.OutputField(desc="Right, Left, or Center")
    party_output = dspy.Predict(classify_party)
    party = party_output(text=text).toDict()['party']
    
    
    return {"party": party, "articles": message_content}

In [140]:
print(text_to_politics("January 6th was a dark day for America. We must never forget the insurrection."))

{'party': 'Left', 'articles': '{\n  "articles": [{\n    "title": "How Different Accounts of the Riots in Washington D.C. Paint a Daunting Picture of America",\n    "link": "https://www.theatlantic.com/magazine/archive/2021/02/how-different-accounts-of-the-riots-in-washington-d-c-paint-a-daunting-picture-of-america/618133/"\n  }, {\n    "title": "U.S. House Science Committee Chairman Holds Hearing on Assault on U.S. Capitol",\n    "link": "https://science.house.gov/latest-news/husc-chairman-holds-field-hearing-assault-on-u-s-capitol"\n  }, {\n    "title": "Giuliani,, Trump\'s Attempts to Overturn Vote Are Rejected by 300",\n    "link": "https://twitter.com/realDonaldTrump/status/1355416824902403073"\n  }]\n}'}


In [103]:
turbo = dspy.OpenAI(model='gpt-4', api_key=OPENAI_API_KEY)
dspy.configure(lm=turbo)

In [132]:
text = """
Here are the top three links to current news articles related to the tweet: 

```json
[{
    "title": "U.S. Capitol Riots: One Year Later, Reliving That Dark Day for America",
    "link": "https://www.npr.org/2022/01/06/1054647356/us-capitol-riots-one-year-later-reliving-that-dark-day-for-america"
}, {
    "title": "One Year Later, A Look Into The Unforeseen Consequences Of January 6th",
    "link": "https://www.forbes.com/sites/madelinebergman/2022/01/06/one-year-later-a-look-into-the-unforeseen-consequences-of-january-6th/?sh=7f50fa5b7b54"
}, {
    "title": "Bidenamazonaws.com",
    "link": "https://www.presidency.ucsb.edu/node/349507"
}]
```
"""
class format_output(dspy.Signature):
    """Given a string representation of a json object, remove the top key and just return a list of json objects."""
    text = dspy.InputField()
    party = dspy.OutputField(desc="list of json objects")
party = dspy.Predict(format_output)
answer = party(text=text)
answer = answer.toDict()['party']
obj = json.dumps(answer)
obj

class classify_party(dspy.Signature):
    """Given tweet text, classify the political party of the author"""
    text = dspy.InputField()
    party = dspy.OutputField(desc="Right, Left, or Center")
party = dspy.Predict(classify_party)
answer = party(text="January 6th was a dark day for America. We must never forget the insurrection.")
answer.toDict()['party']



'Left'